# Milestone 2 - Tool Comparison
## Name: Alexander James, Joshua Ludolf, & Matthew Trevino
``Date: 02-25-2025``
### Description of this file:
This Jupyter Notebook provides a comprehensive walkthrough of using the `NFStream`, `Pyshark` & `Scapy`  library for network traffic analysis. Additionally noting Strengths & Weaknesses. The notebook includes the following sections:

1. **Installation of Requirements**: Installing the necessary `NFStream`, `Pyshark` & `Scapy` library.
2. **Importing Libraries**: Importing the required libraries for network traffic analysis.
3. **Strengths & Weaknesses**: Detailed information about the strengths and weaknesses.

In [1]:
%pip install nfstream
%pip install scapy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.




## NFStream

### Strengths
- **Performance**: Highly optimized C++ core with Python bindings makes NFStream significantly faster for processing large pcap files or real-time traffic
- **Memory Efficiency**: Uses streaming approach that minimizes memory footprint compared to Scapy and PyShark
- **Flow-based Analysis**: Automatically handles flow creation and tracking, providing higher-level abstractions
- **Built-in Feature Extraction**: Provides 100+ pre-built traffic features out of the box
- **ML Integration**: Designed to work well with machine learning pipelines

### Weaknesses
- **Less Flexibility**: Offers fewer packet manipulation capabilities compared to Scapy
- **Learning Curve**: Flow-based paradigm may require adjustment for those used to packet-level analysis
- **Limited Protocol Support**: Supports fewer application protocols than PyShark (which leverages Wireshark dissectors)
- **Python Version**: Only operates in python 3.11 or 3.8 and below (other version failed in build on github)

In [ ]:
from nfstream import NFStreamer

# Use the Intel(R) Wi-Fi 6 AX201 160MHz interface name for pc (found using `ipconfig` or `ip addr` and then copying description of interface as source)
streamer = NFStreamer(source="Intel(R) Wi-Fi 6 AX201 160MHz")

for flow in streamer:
    print(f"Source IP: {flow.src_ip}, Destination IP: {flow.dst_ip}, Protocol: {flow.protocol}, Bytes: {flow.bidirectional_bytes}")

Source IP: 192.168.1.199, Destination IP: 224.0.0.251, Protocol: 17, Bytes: 1707
Source IP: fe80::9168:1ca9:dd4d:255c, Destination IP: ff02::fb, Protocol: 17, Bytes: 1827
Source IP: 192.168.1.199, Destination IP: 192.168.1.1, Protocol: 17, Bytes: 1485
Source IP: 52.109.0.91, Destination IP: 192.168.1.199, Protocol: 6, Bytes: 309
Source IP: 192.168.1.199, Destination IP: 140.82.113.22, Protocol: 6, Bytes: 442
Source IP: 192.168.1.199, Destination IP: 13.107.138.10, Protocol: 6, Bytes: 420761
Source IP: 192.168.1.199, Destination IP: 52.188.247.148, Protocol: 6, Bytes: 7925
Source IP: 13.107.246.57, Destination IP: 192.168.1.199, Protocol: 6, Bytes: 333
Source IP: 192.168.1.199, Destination IP: 140.82.113.22, Protocol: 6, Bytes: 442
Source IP: 192.168.1.199, Destination IP: 140.82.113.22, Protocol: 6, Bytes: 442
Source IP: 140.82.113.22, Destination IP: 192.168.1.199, Protocol: 6, Bytes: 443
Source IP: 192.168.1.199, Destination IP: 20.189.173.25, Protocol: 6, Bytes: 272
Source IP: fe80:

## Scapy

### Strengths
- **Performance**: Significantly faster than NFStream for large-scale analysis
- **Packet Creation/Manipulation**: Unmatched flexibility for crafting custom packets and protocols
- **Interactive Use**: Excellent for testing and experimenting with network protocols
- **Powerful Dissection**: Can decode a wide range of protocols with manual control
- **Scriptability**: Great for automating complex network tasks and penetration testing

### Weaknesses
- **Memory Usage**: Loads entire packet captures into memory
- **Steep Learning Curve**: Requires deep knowledge of protocol structures
- **Limited Scalability**: Not ideal for processing gigabytes of traffic data

In [2]:
from scapy.all import IP, TCP, send, sniff

# Create a custom packet
packet = IP(dst="192.168.1.1")/TCP(dport=80)/"GET / HTTP/1.1\r\nHost: 192.168.1.1\r\n\r\n"

# Send the packet
send(packet)

# Sniff and analyze packets
def packet_callback(packet):
    if packet.haslayer(TCP):
        print(f"Packet: {packet.summary()}")

sniff(iface="Intel(R) Wi-Fi 6 AX201 160MHz", prn=packet_callback, count=10)


Sent 1 packets.
Packet: Ether / IP / TCP 192.168.1.1:http > 192.168.1.199:ftp_data SA
Packet: Ether / IP / TCP 192.168.1.199:64527 > 52.96.121.162:https A / Raw
Packet: Ether / IP / TCP 52.96.121.162:https > 192.168.1.199:64527 A
Packet: Ether / IP / TCP 192.168.1.199:49411 > 20.7.1.246:https A / Raw
Packet: Ether / IP / TCP 20.7.1.246:https > 192.168.1.199:49411 A
Packet: Ether / IP / TCP 192.168.1.1:http > 192.168.1.199:ftp_data SA
Packet: Ether / IP / TCP 52.182.143.208:https > 192.168.1.199:65324 PA / Raw
Packet: Ether / IP / TCP 52.182.143.208:https > 192.168.1.199:65324 FA
Packet: Ether / IP / TCP 192.168.1.199:65324 > 52.182.143.208:https A
Packet: Ether / IP / TCP 192.168.1.199:65324 > 52.182.143.208:https PA / Raw


<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>

## PyShark - See/execute shark.py for sample

### Strengths
- **Wireshark Integration**: Access to all Wireshark dissectors for comprehensive protocol support
- **Readable Output**: Human-friendly packet information similar to Wireshark GUI
- **Familiar Interface**: Easy transition for Wireshark users
- **Deep Packet Inspection**: Excellent for detailed protocol analysis

### Weaknesses
- **Performance**: Much slower than NFStream, relies on tshark processes
- **Resource Intensive**: High memory usage when dealing with large captures
- **Dependency on Wireshark**: Requires Wireshark/tshark installation
- **Limited Packet Creation**: Not designed for packet crafting like Scapy
- **Jupyter Notebook Limitations**: Often encounters execution issues in Jupyter Notebook 
        environments due to asynchronous processing requirements